# Feature Engineering on a Financial Dataset

<b> Download and load the different tables from this dataset </b>

In [1]:
# import packages
import pandas as pd
import numpy as np

In [2]:
# url paths
disp_url = 'https://raw.githubusercontent.com/PacktWorkshops/The-Data-Science-Workshop/master/Chapter12/Dataset/disp.csv'
trans_url = 'https://raw.githubusercontent.com/PacktWorkshops/The-Data-Science-Workshop/master/Chapter12/Dataset/trans.csv'
account_url = 'https://raw.githubusercontent.com/PacktWorkshops/The-Data-Science-Workshop/master/Chapter12/Dataset/account.csv'
client_url = 'https://raw.githubusercontent.com/PacktWorkshops/The-Data-Science-Workshop/master/Chapter12/Dataset/client.csv'

In [3]:
# load the data
df_disp = pd.read_csv(disp_url, sep=';')
df_trans = pd.read_csv(trans_url, sep=';')
df_account = pd.read_csv(account_url, sep=';')
df_client = pd.read_csv(client_url, sep=';')

C:\Users\Hevans\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<b> Analyze each table with the .shape and .head() methods </b>

In [4]:
print(df_disp.shape)
print(df_trans.shape)
print(df_account.shape)
print(df_client.shape)

(5369, 4)
(1056320, 10)
(4500, 4)
(5369, 3)


In [5]:
df_disp.head()

,disp_id,client_id,account_id,type
0,1,1,1,OWNER
1,2,2,2,OWNER
2,3,3,2,DISPONENT
3,4,4,3,OWNER
4,5,5,3,DISPONENT


In [6]:
df_trans.head()

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
0,695247,2378,930101,PRIJEM,VKLAD,700.0,700.0,NaN,NaN,NaN
1,171812,576,930101,PRIJEM,VKLAD,900.0,900.0,NaN,NaN,NaN
2,207264,704,930101,PRIJEM,VKLAD,1000.0,1000.0,NaN,NaN,NaN
3,1117247,3818,930101,PRIJEM,VKLAD,600.0,600.0,NaN,NaN,NaN
4,579373,1972,930102,PRIJEM,VKLAD,400.0,400.0,NaN,NaN,NaN


In [7]:
df_account.head()

,account_id,district_id,frequency,date
0,576,55,POPLATEK MESICNE,930101
1,3818,74,POPLATEK MESICNE,930101
2,704,55,POPLATEK MESICNE,930101
3,2378,16,POPLATEK MESICNE,930101
4,2632,24,POPLATEK MESICNE,930102


In [8]:
df_client.head()

,client_id,birth_number,district_id
0,1,706213,18
1,2,450204,1
2,3,406009,1
3,4,561201,5
4,5,605703,5


<b> Find the common/similar column(s) between tables that will be used for merging based on the analysis from Step 2 </b>

<b> There should be four common tables. Merge the four tables together </b>

In [9]:
# Merge df_trans and df_account together using left join on the account_id column
df_trans_acc = pd.merge(df_trans, df_account, how='left', on='account_id')

In [10]:
#Subset df_disp to only keep the rows with OWNER as their type
df_disp_owner = df_disp[df_disp['type'] == 'OWNER']

In [11]:
# Merge df_trans_acc and df_disp_owner together using left join on the account_id column
df_trans_acc_disp = pd.merge(df_trans_acc, df_disp, how='left', on='account_id')

In [12]:
df_merged = pd.merge(df_trans_acc_disp, df_client, how='left', on=['client_id', 'district_id'])

<b> Rename the column names after merging </b>

In [13]:
df_merged.columns

Index(['trans_id', 'account_id', 'date_x', 'type_x', 'operation', 'amount',
       'balance', 'k_symbol', 'bank', 'account', 'district_id', 'frequency',
       'date_y', 'disp_id', 'client_id', 'type_y', 'birth_number'],
      dtype='object')

In [14]:
df_merged.rename(columns={'date_x': 'trans_date', \
                          'type_x': 'trans_type', \
                          'date_y':'account_creation', \
                          'type_y':'client_type'}, \
                          inplace=True)

In [15]:
df_merged.head()

,trans_id,account_id,trans_date,trans_type,operation,amount,balance,k_symbol,bank,account,district_id,frequency,account_creation,disp_id,client_id,client_type,birth_number
0,695247,2378,930101,PRIJEM,VKLAD,700.0,700.0,NaN,NaN,NaN,16,POPLATEK MESICNE,930101,2873,2873,OWNER,755324.0
1,171812,576,930101,PRIJEM,VKLAD,900.0,900.0,NaN,NaN,NaN,55,POPLATEK MESICNE,930101,692,692,OWNER,NaN
2,171812,576,930101,PRIJEM,VKLAD,900.0,900.0,NaN,NaN,NaN,55,POPLATEK MESICNE,930101,693,693,DISPONENT,NaN
3,207264,704,930101,PRIJEM,VKLAD,1000.0,1000.0,NaN,NaN,NaN,55,POPLATEK MESICNE,930101,844,844,OWNER,NaN
4,207264,704,930101,PRIJEM,VKLAD,1000.0,1000.0,NaN,NaN,NaN,55,POPLATEK MESICNE,930101,845,845,DISPONENT,NaN


<b> Check there is no duplication after merging </b>

<b> Transform the data type for date columns </b>

In [16]:
df_merged.dtypes

trans_id              int64
account_id            int64
trans_date            int64
trans_type           object
operation            object
amount              float64
balance             float64
k_symbol             object
bank                 object
account             float64
district_id           int64
frequency            object
account_creation      int64
disp_id               int64
client_id             int64
client_type          object
birth_number        float64
dtype: object

In [17]:
df_merged['trans_date'] = pd.to_datetime(df_merged['trans_date'], format="%y%m%d")
df_merged['account_creation'] = pd.to_datetime(df_merged['account_creation'], format="%y%m%d")

<b> Create two separate features from birth_number to get the date of birth and sex for each customer </b>

Here is the rule used for this code: birthday and sex: the number is in the form YYMMDD for men, the number is in the form YYMM+50DD for women, where YYMMDD is the date of birth.

In [18]:
df_merged['is_female'] = (df_merged['birth_number'] % 10000) / 5000 > 1

In [19]:
# Transform all the rows with is_female is True by removing the value within the column birth_number by 5000
df_merged.loc[df_merged['is_female'] == True, 'birth_number'] -= 5000

In [20]:
pd.to_datetime(df_merged['birth_number'], format="%y%m%d", errors='coerce')

0         1975-03-24
1                NaT
2                NaT
3                NaT
4                NaT
             ...    
1262620   2066-11-01
1262621   2033-12-31
1262622   2022-07-20
1262623   2018-02-05
1262624   2046-12-02
Name: birth_number, Length: 1262625, dtype: datetime64[ns]

In [21]:
df_merged['birth_number'] = df_merged['birth_number'].astype(str)

In [22]:
df_merged.loc[df_merged['birth_number'] == 'nan', 'birth_number'] = np.nan

<b> Fix data quality issues with .isna() </b>

In [23]:
# Add the 19 prefix to birth_number for all rows that don't have missing values for this column
df_merged.loc[~df_merged['birth_number'].isna(), 'birth_number'] = \
'19' + df_merged.loc[~df_merged['birth_number'].isna(), 'birth_number']

In [24]:
df_merged['birth_number'] = pd.to_datetime(df_merged['birth_number'], format="%Y%m%d", errors='coerce')

<b> Create a new feature that will calculate customers' ages when they opened an account using date operations </b>

In [25]:
df_merged['age_at_creation'] = df_merged['account_creation'] - df_merged['birth_number']

In [26]:
df_merged['age_at_creation'] = df_merged['age_at_creation'] / np.timedelta64(1,'Y')

In [27]:
df_merged['age_at_creation'] = df_merged['age_at_creation'].round()